# INVENTORY PLANNING B

In [ ]:
library(plyr)

We start again with some basic information like cost data

In [ ]:
u_cost <- 7.50
u_price <- 10.00
u_refund <- 2.50
order_quantity <- 200

and demand distribution.

In [ ]:
thou_samples <- sample(c(100, 150, 200, 250, 300), size = 1000, replace = TRUE, prob = c(0.30, 0.20, 0.30, 0.15, 0.05))
#be careful, demands and probabilities should be written respectively

Now we can create an empty data frame which will get 1000 replications and their corresponding outputs.

In [ ]:
inv_plan_table_thou <- data.frame(matrix(nrow = 1000, ncol = 6))
colnames(inv_plan_table_thou) <- c("Replication", "Demand", "Revenue", "Cost", "Refund", "Profit")

It is time to replicate:

In [ ]:
for(i in 1:1000){
      inv_plan_table_thou[i,1] <- i #replication number
      inv_plan_table_thou[i,2] <- thou_samples[i] #demand derived from demand distribution
      inv_plan_table_thou[i,3] <- u_price*min(order_quantity, inv_plan_table_thou[i,2]) #revenue
      inv_plan_table_thou[i,4] <- u_cost*order_quantity #cost
      inv_plan_table_thou[i,5] <- if(order_quantity > inv_plan_table_thou[i,2]){u_refund*(order_quantity - inv_plan_table_thou[i,2])} else {0} #refund
      inv_plan_table_thou[i,6] <- inv_plan_table_thou[i,3] - inv_plan_table_thou[i,4] + inv_plan_table_thou[i,5] #profit
}

inv_plan_table_thou

After that, we can get several other information from this data. Average profit, standard deviation of profit, and min&max profit for instance.

In [ ]:
avg_profit <- mean(inv_plan_table_thou[,6])
sd_profit <- sd(inv_plan_table_thou[,6])
min_profit <- min(inv_plan_table_thou[,6])
max_profit <- max(inv_plan_table_thou[,6])

print(c(avg_profit, sd_profit, min_profit, max_profit))

95% confidence interval for expected profit is calculated using sd_profit variable above:

In [ ]:
error <- qnorm(0.975)*sd_profit/sqrt(1000)
lower_limit <- avg_profit - error
upper_limit <- avg_profit + error

print(c(lower_limit, upper_limit))

We can count distribution of profit:

In [ ]:
table(inv_plan_table_thou[,6]) #sixth column of table is profit

To calculate average profit per order quantity, first we need to determine order quantities and put them in a table:

In [ ]:
order_quantity_diff <- c(seq(from=100, to=300, by=25))
avg_prof_vs_order_quan <- data.frame(matrix(ncol = 2, nrow = length(order_quantity_diff)))
colnames(avg_prof_vs_order_quan) <- c("order quantity", "average profit")

Now, we can replicate again WITH A DIFFERENCE. We will calculate changing average profits per order quantity.

In [ ]:
t <- 0

for(j in order_quantity_diff) #in order quantities
{
  for(i in 1:1000){ #in replications
    inv_plan_table_thou[i,1] <- i
    inv_plan_table_thou[i,2] <- thou_samples[i]
    inv_plan_table_thou[i,3] <- u_price*min(j, inv_plan_table_thou[i,2])
    inv_plan_table_thou[i,4] <- u_cost*j
    inv_plan_table_thou[i,5] <- if(j > inv_plan_table_thou[i,2]){u_refund*(j - inv_plan_table_thou[i,2])} else {0}
    inv_plan_table_thou[i,6] <- inv_plan_table_thou[i,3] - inv_plan_table_thou[i,4] + inv_plan_table_thou[i,5]
  }
  t <- t + 1
  avg_prof_vs_order_quan[t,] <- c(j, mean(inv_plan_table_thou[,6])) #assign them into the table we've created above
}

avg_prof_vs_order_quan #check this out

## Let's repeat this for one-way table (easy compared to two-way):

In [ ]:
for(i in 1:1000){
  inv_plan_table_thou[i,1] <- i
  inv_plan_table_thou[i,2] <- thou_samples[i]
  inv_plan_table_thou[i,3] <- u_price*min(order_quantity, inv_plan_table_thou[i,2])
  inv_plan_table_thou[i,4] <- u_cost*order_quantity
  inv_plan_table_thou[i,5] <- if(order_quantity > inv_plan_table_thou[i,2]){u_refund*(order_quantity - inv_plan_table_thou[i,2])} else {0}
  inv_plan_table_thou[i,6] <- inv_plan_table_thou[i,3] - inv_plan_table_thou[i,4] + inv_plan_table_thou[i,5]
}

one_way_dt <- inv_plan_table_thou[,c(1,6)]
one_way_dt

## Two-way table:

We need to create a new table for two-way which will use various order quantities:

In [ ]:
two_way_dt <- data.frame(matrix(nrow = 1000, ncol = length(order_quantity_diff)))
colnames(two_way_dt) <- c(order_quantity_diff)

We will repeat same steps as other calculations but now we use only "profits" column and extract them into our new "two-way table".

In [ ]:
t <- 0
for(j in order_quantity_diff)
{
  for(i in 1:1000){
    inv_plan_table_thou[i,1] <- i
    inv_plan_table_thou[i,2] <- thou_samples[i]
    inv_plan_table_thou[i,3] <- u_price*min(j, inv_plan_table_thou[i,2])
    inv_plan_table_thou[i,4] <- u_cost*j
    inv_plan_table_thou[i,5] <- if(j > inv_plan_table_thou[i,2]){u_refund*(j - inv_plan_table_thou[i,2])} else {0}
    inv_plan_table_thou[i,6] <- inv_plan_table_thou[i,3] - inv_plan_table_thou[i,4] + inv_plan_table_thou[i,5]
  }
  t <- t + 1
  two_way_dt[,t] <- inv_plan_table_thou[,6]
}

two_way_dt

We can calculate summary measures too:

In [ ]:
sum_measures <- matrix(nrow = 2, ncol = length(order_quantity_diff))
for(y in 1:length(order_quantity_diff))
{
 sum_measures[1,y] <- mean(two_way_dt[,y]) #means of profits for order quantities
 sum_measures[2,y] <- sd(two_way_dt[,y]) #standard deviations of profits for order quantities
}

colnames(sum_measures) <- c(order_quantity_diff)
rownames(sum_measures) <- c("mean", "sd")

sum_measures

For extra style points, we can compare average profits for order quantities and convert this intro barplot:

In [ ]:
barplot(sum_measures[1,], names.arg = order_quantity_diff, 
        ylab = "Average Profit", xlab = "Order Quantity", main = "Average Profit versus Order Quantity")

### Analytical Solution

In [ ]:
mcu <- u_price - u_cost #marginal cost of understock
mco <- u_cost - u_refund #marginal cost of overstock
mcu/(mcu + mco)